In [32]:
import requests
import pandas as pd
import numpy as np
import time
import pprint as pp
from tqdm.notebook import tqdm
import time
import psutil
import sys

ModuleNotFoundError: No module named 'tqdm.notebook'

In [6]:
#Collect zip codes and categories
zip_codes = [line.rstrip('\n') for line in open('zips.txt')]
categories = [line.rstrip('\n') for line in open('categories.txt')]

In [7]:
#Define the API Key, Endpoint, and Header
API1=
API2=
API3=
API4=
ENDPOINT = 'https://api.yelp.com/v3/businesses/search'
HEADERS = {'Authorization': 'bearer %s' % API1}

In [27]:
#Initialize lists for collecting data
name = []
url = []
phone = []
address = []
city = []
state = []
czip = []
rating = []
review_count = []
vertical = []
search = []
yelp_id = []
review1 = []
review2 = []
review3 = []

In [9]:
#Ask user for page number to collect from
print('Page Number:')
page_number = int(input())
page_number = (page_number-1)*50

Page Number:
1


In [31]:
count = 0
for page in tqdm([0,50,100,150]):
    count+=1
    #try:
    PARAMETERS = {'location':'san diego', 
              'term': 'italian',
              'limit': 50,
              'offset':page}
    if 5000<=count<=9999:
        HEADERS = {'Authorization': 'bearer %s' % API2}
    if 10000<=count<=14999:
        HEADERS = {'Authorization': 'bearer %s' % API3}
    if 15000<=count<=19999:
        HEADERS = {'Authorization': 'bearer %s' % API4}
    if 20000<=count<=24999:
        HEADERS = {'Authorization': 'bearer %s' % API1}
    if 25000<=count:
        print('Used up all API requests available')
        pass
    response = requests.get(url = ENDPOINT, params = PARAMETERS, headers = HEADERS)
    if response.status_code!=200:
        print('Error: response status code not 200')
        time.sleep(5)
        sys.exit(0)
    business_data = response.json()
    for business in tqdm(business_data['businesses']):
        name = name + [business['name']]
        url = url + [business['url']]
        address = address + [business['location']['address1']]
        city = city + [business['location']['city']]
        state = state + [business['location']['state']]
        czip = czip + [business['location']['zip_code']]
        phone = phone + [business['phone']]
        #search = search + [cat]
        yelp_id = yelp_id + [business['id']]
        industries = str()
        for i in business['categories']:
            industries = industries+', '+str(i['title'])
        vertical = vertical + [industries[2:]]
        rating = rating + [business['rating']]
        review_count = review_count + [business['review_count']]
        # Reviews
        response_reviews = requests.get(url = 'https://api.yelp.com/v3/businesses/'+business['id']+'/reviews',
            params = {}, headers = HEADERS)
        review_data = response_reviews.json()
        try:
            review1 += [review_data['reviews'][0]['text']]
        except: review1 += ['null']
        try:
            review2 += [review_data['reviews'][1]['text']]
        except: review2 += ['null']
        try:
            review3 += [review_data['reviews'][2]['text']]
        except: review3 += ['null']
#     except:
#         print('error')

100%|██████████| 4/4 [05:55<00:00, 84.93s/it]t]


In [48]:
len(vertical)

200

In [49]:
result = pd.DataFrame({
    'Name':name[:200],
    'URL':url[:200],
    'Phone':phone[:200],
    'Address':address[:200],
    'City':city[:200],
    'State':state[:200],
    'ZIP':czip[:200],
    'Vertical':vertical[:200],
    'Rating':rating[:200],
    'Review Count':review_count[:200],
    'Yelp ID':yelp_id[:200]
})
print('Original length of collected data: '+str(len(result))+' listings.')

Original length of collected data: 200 listings.


In [64]:
def vertical_split(x_):
    first = []
    second = []
    third = []
    for _ in x_:
        x = _.split(', ')
        try:
            first += [x[0]]
        except: first += ['None']
        try:
            second += [x[1]]
        except: second += ['None']
        try:
            third += [x[2]]
        except: third += ['None']
    return first, second, third

In [65]:
result['Vertical1'], result['Vertical2'], result['Vertical3'] = vertical_split(result['Vertical'])

In [67]:
len(result)

200

In [69]:
result.drop_duplicates(subset=['Phone'], inplace=True)

In [71]:
result = pd.get_dummies(result, columns=['City','ZIP','Vertical1','Vertical2','Vertical3'], drop_first=True)

In [75]:
result['Ranking'] = list(range(1,len(result)+1))

In [77]:
result_ = result[['Name','URL','Phone','Address','State','Vertical','Yelp ID']]
result = result.drop(['Name','URL','Phone','Address','State','Vertical','Yelp ID'],axis=1)

In [79]:
result.head()

,Rating,Review Count,City_Cardiff By The Sea,City_Chula Vista,City_Coronado,City_Del Mar,City_El Cajon,City_Encinitas,City_Imperial Beach,City_La Jolla,...,Vertical3_Sandwiches,Vertical3_Seafood,Vertical3_Sicilian,Vertical3_Soul Food,Vertical3_Soup,Vertical3_Tapas Bars,Vertical3_Vegan,Vertical3_Venues & Event Spaces,Vertical3_Wine Bars,Ranking
0,4.5,575,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,4.5,2419,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,4.5,974,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,4.0,2505,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
10,4.0,814,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,5


In [81]:
result.head()

,Rating,Review Count,City_Cardiff By The Sea,City_Chula Vista,City_Coronado,City_Del Mar,City_El Cajon,City_Encinitas,City_Imperial Beach,City_La Jolla,...,Vertical3_Sandwiches,Vertical3_Seafood,Vertical3_Sicilian,Vertical3_Soul Food,Vertical3_Soup,Vertical3_Tapas Bars,Vertical3_Vegan,Vertical3_Venues & Event Spaces,Vertical3_Wine Bars,Ranking
0,4.5,575,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,4.5,2419,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,4.5,974,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,4.0,2505,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
10,4.0,814,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,5


In [95]:
import matplotlib.pyplot as plt  

In [99]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X = result.drop('Ranking',axis=1)
y = result.Ranking
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [100]:
linreg = LogisticRegression().fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [101]:
#To retrieve the intercept:
print(linreg.intercept_)
#For retrieving the slope:
print(linreg.coef_)

[-1.43317660e-02 -2.23649694e-01 -7.20380431e-02 -2.31366206e-01
 -5.05021448e-02  3.29326739e-02  1.50059276e-02  2.16590230e-03
 -4.36384642e-02 -1.40207810e-01  2.10479906e-02 -2.59382149e-01
  6.68907019e-03  1.01369773e-01 -3.11674758e-02 -2.64845872e-02
  5.98297021e-02  4.22674044e-02 -2.76865838e-02 -1.84329320e-01
  2.18178802e-02  1.24248189e-02  1.68247394e-02 -4.14414768e-02
 -1.01635620e-01 -5.75477561e-02  2.38436637e-02  8.02102246e-02
  4.53078080e-02 -1.15026616e-01 -8.04371838e-02  1.02739417e-02
  8.99197500e-04  4.79112051e-02 -5.36593198e-02  2.36699287e-03
 -4.46977445e-02 -1.01260218e-01  5.42882410e-02 -3.22311880e-02
  2.02207659e-02 -3.09738990e-02  4.30109342e-02 -6.94555017e-03
  6.40481733e-03  5.03675704e-02 -3.49255017e-02  3.62518314e-02
  4.46898561e-02 -1.25747804e-01  5.85884652e-02 -2.03479507e-02
 -2.16837036e-02 -1.09106390e-02  7.87073232e-02  7.14740680e-02
  6.42889757e-02  4.17044976e-02  3.94188695e-02 -2.69601993e-03
  3.89555765e-02 -3.20254

In [102]:
y_pred = linreg.predict(X_test)

In [103]:
df = pd.DataFrame({'Actual': np.array(y_test).flatten(), 'Predicted': y_pred.flatten()})
df.head()

,Actual,Predicted
0,185,71
1,164,96
2,19,78
3,16,129
4,68,143


In [104]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 63.25
Mean Squared Error: 5613.666666666667
Root Mean Squared Error: 74.92440634844341
